# CVA Calculation for Risky Bond
This exercise is taken from [HERE](https://www.finrgb.com/swatches/cva-calculation-for-risky-bond-solved-example/)
Credit: Bhuvnesh Khurana

Question:
A 3-year corporate bond pays a coupon of 8% per annum (once a year). The term structure of risk-free interest rates is flat at 4% p.a. and is expected to stay unchanged in the future. If the risk-neutral conditional probability of default of the issuer is 2% for a 1-year period and the recovery rate in event of default is 40% (assumed constant), the credit spread of this bond is CLOSEST to:
A) 120 bps
B) 125 bps
C) 130 bps
D) 135 bps

In [2]:
import numpy as np

In [11]:
PD

[0.02, 0.0196, 0.0192]

In [18]:
EE = [115.5444, 111.8462, 108]
PD = [0.02, 0.0196, 0.0192]
DF = [0.96153, 0.9246, 0.8890]
LGD = 0.6

CVA = LGD*sum(x*y*z for x, y, z in zip(EE,PD,DF))
CVA

3.6553879622592